This repository has Python classes for iterating through the annotations
in the [Buckeye Corpus](http://buckeyecorpus.osu.edu/). They include 
cross-references between the .words, .phones, and .log files, and
can be used to extract sound clips from the .wav files. The docstrings in
`buckeye.py` and `containers.py` describe how to use the classes in more
detail.

The scripts can be installed directly from GitHub with pip using this command:

    pip install git+http://github.com/scjs/buckeye.git

You can also copy the `buckeye/` subdirectory into your working directory, or
put it in your Python path.

### Speaker

A `Speaker` instance is created by pointing to one of the zipped speaker
archives that can be downloaded from the corpus website.

In [1]:
import buckeye

speaker = buckeye.Speaker('./speakers/s01.zip')

This will open and process the annotations in each of the sub-archives inside
the speaker archive (the tracks, such as `s0101a` and `s0101b`). If an optional
`load_wavs` argument is set to `True` when creating a `Speaker` instance, the
wav files associated with each track will also be loaded into memory. Otherwise,
only the annotations are loaded.

Each `Speaker` instance has the speaker's code-name, sex, age, and interviewer
sex available as attributes.

In [2]:
print speaker.name, speaker.sex, speaker.age, speaker.interviewer

s01 f y f


The tracks can be accessed by iterating through the `Speaker` instance.
There is more detail about accessing the annotations below under the heading
**Tracks**.

In [3]:
for track in speaker:
    print track.name

s0101a
s0101b
s0102a
s0102b
s0103a


The tracks can also be accessed through the `tracks` attribute.

In [4]:
print speaker.tracks

[Track("s01/s0101a.zip"), Track("s01/s0101b.zip"), Track("s01/s0102a.zip"), Track("s01/s0102b.zip"), Track("s01/s0103a.zip")]


### Track

Each `Track` instance has `words`, `phones`, `log`, `txt`, and `wav` attributes that contain the corpus data from one track for one speaker. Each speaker has 6 or so tracks.

In [5]:
speaker

Speaker("./speakers/s01.zip")

In [6]:
track = speaker[0]
track

Track("s01/s0101a.zip")

#### Words

The `words` attribute stores a list of Word and Pause instances, created from the `.words` file.

In [7]:
track.words[:10]

[Pause('{B_TRANS}', 0.0, 0.102385),
 Pause('<SIL>', 0.102385, 4.275744),
 Pause('<NOISE>', 4.275744, 8.513518),
 Pause('<IVER>', 8.513518, 32.216575),
 Word('okay', 32.216575, 32.622045, ['ow', 'k', 'ey'], ['k', 'ay'], 'NN'),
 Pause('<IVER>', 32.622045, 37.129002),
 Pause('<VOCNOISE>', 37.129002, 38.123014),
 Pause('<IVER>', 38.123014, 44.617996),
 Word('um', 44.617996, 44.946848, ['ah', 'm'], ['ah', 'm'], 'UH'),
 Pause('<SIL>', 44.946848, 45.355708)]

Word instances have these nine attributes:

* `orthography` (the word's spelling)
* `beg` (word onset time, in seconds)
* `end` (word offset time)
* `dur` (duration)
* `phonemic` (the canonical transcription)
* `phonetic` (the close transcription)
* `pos` (the word's part of speech)
* `misaligned` (marked as True if the word has a negative duration, or if the phonetic transcription doesn't match what's in the `.phones` file)
* `phones` (a list of references to Phone instances that have the labels and timing information for the phonetic transcription)

In [8]:
word = track.words[4]

print word.orthography, word.beg, word.end, word.dur, word.phonemic, word.phonetic, word.pos, word.misaligned

okay 32.216575 32.622045 0.40547 ['ow', 'k', 'ey'] ['k', 'ay'] NN False


The phones are retrieved based on the timestamps for the word and for the entries in the `.phones` file.

In [9]:
word.phones

[Phone('k', 32.216575, 32.376593), Phone('ay', 32.376593, 32.622045)]

Phones have four attributes:

* `seg` (the pseudo-ARPABET transcription of the phone)
* `beg` (onset time)
* `end` (offset time)
* `dur` (duration)

In [10]:
for phone in word.phones:
    print phone.seg, phone.beg, phone.end, phone.dur

k 32.216575 32.376593 0.160018
ay 32.376593 32.622045 0.245452


Many of the annotations are things like `<SIL>` (silence) or `<IVER>` (the interviewer's turn). These are yielded as Pause instances, not as Word instances. Pause instances have six attributes:

* `entry` (the kind of Pause, e.g. `<SIL>`)
* `beg` (pause onset time, in seconds)
* `end` (pause offset time)
* `dur` (duration)
* `misaligned` (marked as True if the Pause has a negative duration)
* `phones` (a list of references to Phone instances that are associated with this Pause, e.g. one or more `SIL` tokens)

In [11]:
pause = track.words[1]

print pause.entry, pause.beg, pause.end, pause.dur, pause.misaligned

<SIL> 0.102385 4.275744 4.173359 False


In [12]:
pause.phones

[Phone('SIL', 0.102385, 4.275744)]

#### Phones

The phones in a track can also be accessed directly through the `phones` attribute of a Track instance.

In [13]:
for phone in track.phones[:10]:
    print phone.seg, phone.beg, phone.end, phone.dur

{B_TRANS} 0.0 0.102385 0.102385
SIL 0.102385 4.275744 4.173359
NOISE 4.275744 8.513763 4.238019
IVER 8.513763 32.216575 23.702812
k 32.216575 32.376593 0.160018
ay 32.376593 32.622045 0.245452
IVER 32.622045 37.129002 4.506957
VOCNOISE 37.129002 38.123014 0.994012
IVER 38.123014 44.617996 6.494982
ah 44.617996 44.820731 0.202735


#### Log

The list of entries in the Track's `.log` file can be accessed through the `log` attribute, which stores a list of the `LogEntry` instances for the Track. `LogEntry` instances have `entry`, `beg`, `end`, and `dur` attributes.

In [14]:
for log in track.log:
    print log.entry, log.beg, log.end, log.dur

<VOICE=modal> 0.0 61.142603 61.142603
<VOICE=creaky> 61.142603 61.397647 0.255044
<VOICE=modal> 61.397647 176.705681 115.308034
<VOICE=creaky> 176.705681 177.442715 0.737034
<VOICE=modal> 177.442715 208.458474 31.015759
<VOICE=creaky> 208.458474 208.998197 0.539723
<IVER_overlap-start> 208.998197 218.326046 9.327849
<IVER_overlap-end> 218.326046 218.619639 0.293593
<IVER_overlap-start> 218.619639 281.4126 62.792961
<IVER_overlap-end> 281.4126 282.015381 0.602781
<VOICE=modal> 282.015381 283.01414 0.998759
<VOICE=creaky> 283.01414 283.342991 0.328851
<IVER_overlap-start> 283.342991 286.3691 3.026109
<IVER_overlap-end> 286.3691 286.587431 0.218331
<IVER_overlap-start> 286.587431 358.243781 71.65635
<IVER_overlap-end> 358.243781 358.766553 0.522772
<VOICE=modal> 358.766553 570.891209 212.124656
<VOICE=creaky> 570.891209 570.988848 0.097639
<IVER_overlap-start> 570.988848 595.595736 24.606888
<IVER_overlap-end> 595.595736 596.178854 0.583118


You can call the `get_logs()` method of a Track to retrieve the log entries that overlap with the given timestamps.

For example, the log entries that overlap with the interval from 60 seconds to 62 seconds can be found like this:

In [15]:
logs = track.get_logs(60.0, 62.0)

for log in logs:
    print log.entry, log.beg, log.end

<VOICE=modal> 0.0 61.142603
<VOICE=creaky> 61.142603 61.397647
<VOICE=modal> 61.397647 176.705681


#### Txt

The `txt` attribute holds a list of speaker turns without timestamps, read from
the `.txt` file in the track.

In [16]:
track.txt[1]

'okay <IVER>'

#### Wavs

If a Speaker instance is created with `load_wavs=True`, each Track will also have a `wav` attribute that stores a `Wave_read` instance.

In [17]:
speaker = buckeye.Speaker('./speakers/s01.zip', load_wavs=True)
track = speaker[0]

track.wav

<wave.Wave_read instance at 0x00000000068FE848>

You can extract sound clips from the wav file with the `clip_wav()` method of each Track:

In [18]:
track.clip_wav('myclip.wav', 60.0, 62.0)

This will create a wav file in the current directory called `myclip.wav` which
contains the sound between 60 and 62 seconds in the track audio.

### Corpus generator

The `corpus()` generator function is a convenience for iterating through all of
the speaker archives together. Put all forty speaker archives in one directory,
such as `speakers/`. Create a new generator with this directory as an argument.

In [19]:
corpus = buckeye.corpus('./speakers')

The generator will yield the `Speaker` instances in numerical order.

In [20]:
for speaker in corpus:
    print speaker.name,

s01 s02 s03 s04 s05 s06 s07 s08 s09 s10 s11 s12 s13 s14 s15 s16 s17 s18 s19 s20 s21 s22 s23 s24 s25 s26 s27 s28 s29 s30 s31 s32 s33 s34 s35 s36 s37 s38 s39 s40


If you're using a `corpus()` generator, you can set `load_wavs` to `True` and it will be passed down to every `Track` instance, so that the all of the wav files will be loaded.

In [21]:
corpus = buckeye.corpus('./speakers', load_wavs=True)